# 1.1 - Gathering FOMC Decision Dates

This script obtains all scheduled FOMC decision dates from February 1994, when The Fed started releasing a statement after every meeting (Gurkaynak, Sack and Swanson, 1994) through to the last date for which Greenbook forecasts are available (accomodating the Mirranda-Aggripino (2016) method of purging shock series of the Fed Information effect).

The script pulls dates from each **federalreserve.gov/monetarypolicy/fomchistorical*yyyy*.htm** page with *yyyy* denoting the year of interest. "Historical" meetings on the Fed website are those for which Greenbook forecasts have been released, so this method attains only those by default. 

Dates are exported to csv.

In [1]:
animals = ["cat", "mouse", "horse"]
animals[0]

'cat'

In [2]:
animals[0:2]

['cat', 'mouse']

### Preamble

This script makes use of...

- `urrlib`
- Regular Expressions (`re`)
- `datetime`
- `time`
- `csv`

In [3]:
import urllib.request
import re
from datetime import datetime
import time
import csv

### Variables for Scraping

In [4]:
# Get current year

current_year = datetime.now().year

# Get list of years

data_years = range(1994, current_year + 1)

# List to which dates appended

dates = []

### Scraping

In [5]:
# Loop

for year in data_years:
    
    year_string = str(year)
    
    url = "http://www.federalreserve.gov/monetarypolicy/fomchistorical" + year_string + ".htm"
    
    try:
        
        xmlstring = str(urllib.request.urlopen(url).read())

    except:
        
        continue
        
         # will fail if no FOMC meetings for which Greenbook released in this year (i.e. the past 5 years or so)
        
    preliminary_matches = re.findall("<h5(.*?)h5>",xmlstring) # Fed website wasn't allowing for standard xml tree parsing
                                                              # so RegEx string parsing used
    
    for match in preliminary_matches:
        
        # Each 'match' is a date
        
        try:
            
            date = re.findall(">(.*?) Meeting", match)[0] # Isolates date component of string
        
        except:
            
            continue
        # Unscheduled meetings contain "(unscheduled)" in string instead of "Meeting", so are avoided by default
        
        if len(date.split()) == 2: # Handles "Month/Month Date-Date" format for overlaps (i.e. May/June 30-1)
        
            months = date.split()[0]
        
            month = months.split('/')[-1] # Takes second month if two
    
            days = date.split()[1]
    
            day = days.split('-')[-1] # Takes last date if multiple
            
            full_date = " ".join([day, month, year_string]) # Concatenates into "1 January 1994" format
            
            ##### BUG FIX #####
            if full_date == "15 September 2003":
                
                print("Skipped " + full_date + """, which is erroneously listed as separate from the 
                                                  Sep 16, 2003 meeting on the Fed website""")
                continue
        
        else:
            # Handles "Month Date-Month Date" format for overlaps (i.e. May 30-June 1)
            
            month_day = date.split("-")[1] # Removes first month & date
            
            month = month_day.split()[0]
            
            day = month_day.split()[1]
            
            full_date = " ".join([day, month, year_string]) # Concatenates into "1 January 1994" format
        
        datetime_date = datetime.strptime(full_date, '%d %B %Y') # Converts String into datetime format
        
        unix_time = time.mktime(datetime_date.timetuple()) # Converts datetime format into Unix time
        
        #unix_time = unix_time + 43200 # Changes Unix time from midnight at start of date to midday of date, to avoid later   
                                      # rounding errors (Times are in UTC).
        
        dates.append(unix_time)

Skipped 15 September 2003, which is erroneously listed as separate from the 
                                                  Sep 16, 2003 meeting on the Fed website


### Export to *.csv*

In [6]:
with open("dates.csv", "w", newline='') as dates_file:
    
    wr = csv.writer(dates_file, quoting=csv.QUOTE_ALL)
    
    wr.writerow(dates)